### Clustering_LDA_4models
TESTA 4 VERSÕES DISTINTAS DE MODELAGEM DE TÓPICOS A PARTIR DE DOCUMENTOS DA BASE NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.

In [1]:
import nltk
import os
import codecs
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
from gensim import corpora, models, similarities #Latent Dirichlet Allocation implementation with Gensim
import pyLDAvis
import pyLDAvis.gensim
from IPython.display import clear_output

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import pickle

C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
C:\Anaconda3\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use 

In [2]:
outputs = "../outputs/"

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro',
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza',

### Seleciona textos da base mysql para fazer modelagem de tópicos

In [4]:
count = 0
texts = []
pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()

with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           #user='marcelobribeiro', 
                           user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    captura documentos da tabela docs para posterior modelagem de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ") # filtra textos   
    data = cur.fetchall()
    numrows = cur.rowcount
    percentil = numrows/100
    
    for row in data:
        count += 1
        if row is None: break

        if count % percentil == 0: 
            clear_output()
            print(int(count/percentil),'% done')

        text =  row[4]
        text = text.split()
        symbols = [x for x in string.punctuation]
        text = [p for p in text if p not in symbols]
        text = [p.strip(string.punctuation) for p in text]
        text = [p for p in text if not p.isdigit()]
        text = [p for p in text if len(p)>1]
        texts.append(text)

········
········


### Cria lista de stopwords

In [5]:
# load nltk's stopwords as variable called 'stopwords'

additional_words = ['mr','one', 'two', 'three', 'four', 
                    'five', 'um', 'dois', 'três', 'quatro', 
                    'cinco', 'janeiro', 'fevereiro', 'março', 
                    'abril', 'maio', 'junho', 'julho', 'agosto', 
                    'setembro', 'outubro', 'novembro', 'dezembro', 
                    'january', 'february', 'march', 'april', 'may', 
                    'june', 'july', 'august', 'september', 
                    'october', 'november', 'december', 'países', 
                    'ser', 'país', 'ainda', 'milhões', 'maior', 
                    'anos', 'grande', 'apenas', 'outros', 'pode', 
                    'parte', 'partes', 'item', 'vossa', 'nota', 
                    'havia', 'pt', 'vg', 'ptvg', 'eh', 'nr', 'hrs', 
                    'pais', 'parte', 'hoje', 'brasemb', 'ontem', 
                    'dia', 'countries', 'would', 'new', 'also', 
                    'must', 'draft', 'shall', 'item', 'page', 
                    'th', 'anos', 'ii', 'dias', 'poderá', 'caso', 
                    'casos', 'qualquer', 'ano', 'mil', 'pessoas', 
                    'único', 'única', 'únicos', 'únicas', 'índice', 
                    'expedido', 'co', 'mm', 'er', 'via', 'ww', 'ra', 
                    'ia', 'ca', 'nu', 'wa', 'aa', 'ms', 'dc', 'mmm', 'pa']

stopwords = nltk.corpus.stopwords.words('english') + \
            nltk.corpus.stopwords.words('portuguese') + \
            nltk.corpus.stopwords.words('french') + \
            nltk.corpus.stopwords.words('spanish') + \
            nltk.corpus.stopwords.words('german') + \
            additional_words

stopwords = list(set(stopwords))

### Latent Dirichlet Allocation

In [49]:
%time texts = [[word for word in text if word not in stopwords] for text in texts]

Wall time: 250 ms


In [50]:
print(len(texts[0]))

32


In [51]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=1, no_above=0.8)
dictionary.filter_tokens(bad_ids=[0,]) #retira palavras a partir do id
corpus = [dictionary.doc2bow(text) for text in texts]

In [52]:
len(corpus)

200

#### Salva dictionary e corpus com o pickle

In [53]:
file_corpus = '../outputs/LDAcorpus.sav'
file_dictionary = '../outputs/LDAdictionary.sav'
pickle.dump(corpus, open(file_corpus, 'wb'))
pickle.dump(dictionary, open(file_dictionary, 'wb'))

### Teste com 20 tópicos.

In [54]:
%time lda = models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=50)

Wall time: 42.2 s


In [55]:
lda.print_topics(-1, num_words=5)

[(0,
  '0.012*silveira + 0.011*gb + 0.008*telegrama + 0.008*azeredo + 0.007*antonio'),
 (1,
  '0.006*silveira + 0.006*azeredo + 0.005*embaixador + 0.004*exteriores + 0.004*lt'),
 (2,
  '0.013*silveira + 0.011*azeredo + 0.010*embaixador + 0.007*vieira + 0.006*zczc'),
 (3, '0.004*lu + 0.004*embratel + 0.004*spo + 0.004*rua + 0.004*caconde'),
 (4,
  '0.009*silveira + 0.008*devo + 0.006*azeredo + 0.006*amigo + 0.005*prezado'),
 (5, '0.010*silveira + 0.006*rio + 0.006*gb + 0.006*souto + 0.006*azeredo'),
 (6,
  '0.011*silveira + 0.010*embaixador + 0.008*df + 0.007*antonio + 0.006*telegrama'),
 (7,
  '0.011*azeredo + 0.009*silveira + 0.008*antonio + 0.008*rio + 0.005*amigo'),
 (8,
  '0.011*silveira + 0.008*azeredo + 0.008*rio + 0.007*exteriores + 0.006*antonio'),
 (9,
  '0.009*silveira + 0.009*embaixador + 0.007*brasil + 0.005*zczc + 0.005*lt'),
 (10,
  '0.013*silveira + 0.011*azeredo + 0.008*antonio + 0.007*souto + 0.007*vieira'),
 (11, '0.017*silveira + 0.016*rio + 0.013*rjo + 0.013*lt + 0.

#### Visualizando os tópicos com o PyLDAvis (20 tópicos)

In [57]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_20topics.html'))

### Teste com 30 tópicos.

In [58]:
%time lda = models.LdaModel(corpus, num_topics=30, id2word=dictionary, passes=50)

Wall time: 44.8 s


In [59]:
lda.print_topics(-1, num_words=5)

[(0, '0.029*rjo + 0.023*241gtx + 0.022*lt + 0.022*rio + 0.020*souto'),
 (1,
  '0.010*azeredo + 0.010*silveira + 0.009*exteriores + 0.009*antonio + 0.008*embaixador'),
 (2,
  '0.011*silveira + 0.009*telegrama + 0.008*embaixador + 0.006*afetuoso + 0.006*azeredo'),
 (3, '0.011*silveira + 0.007*azeredo + 0.007*rio + 0.007*zczc + 0.005*bsb'),
 (4,
  '0.007*silveira + 0.006*vieira + 0.006*telegrama + 0.006*souto + 0.006*ag'),
 (5,
  '0.010*gb + 0.008*embaixador + 0.008*mobral + 0.008*responsável + 0.007*leve'),
 (6,
  '0.010*azeredo + 0.009*embaixador + 0.007*antonio + 0.006*silveira + 0.005*buenos'),
 (7,
  '0.010*antonio + 0.009*azeredo + 0.009*silveira + 0.008*lt + 0.004*souto'),
 (8,
  "0.003*silveira + 0.003*correios + 0.003*horas + 0.003*75303007-0066 + 0.003*u'c"),
 (9, '0.007*vieira + 0.005*bsb + 0.005*brasilia + 0.005*df + 0.005*ministro'),
 (10,
  '0.015*silveira + 0.012*embaixador + 0.011*nome + 0.010*exteriores + 0.008*azeredo'),
 (11, '0.012*silveira + 0.008*rua + 0.007*rio + 0.

#### Visualizando os tópicos com o PyLDAvis (30 tópicos)

In [61]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_30topics.html'))

### Teste com 40 tópicos.

In [ ]:
%time lda = models.LdaModel(corpus, num_topics=40, id2word=dictionary, passes=50)

In [ ]:
lda.print_topics(-1, num_words=5)

#### Visualizando os tópicos com o PyLDAvis (40 tópicos)

In [ ]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [28]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_40topics.html'))

### Teste com 45 tópicos.

In [62]:
%time lda = models.LdaModel(corpus, num_topics=45, id2word=dictionary, passes=50)

Wall time: 48.1 s


In [63]:
lda.print_topics(-1, num_words=5)

[(0, '0.010*rio + 0.007*silveira + 0.007*rjo + 0.007*241gtx + 0.007*vieira'),
 (1,
  '0.020*rio + 0.012*telegrama + 0.011*silveira + 0.009*gb + 0.008*antonio'),
 (2, '0.006*rue + 0.006*geneve + 0.006*aaô + 0.006*éww + 0.006*r)]?ás/l'),
 (3,
  '0.008*azeredo + 0.008*rio + 0.004*designação + 0.004*ama + 0.004*silveira'),
 (4, '0.010*silveira + 0.007*bsb + 0.007*ministro + 0.007*nome + 0.007*grato'),
 (5,
  '0.013*silveira + 0.010*azeredo + 0.009*rio + 0.008*antonio + 0.006*buenos'),
 (6, '0.017*silveira + 0.011*azeredo + 0.009*bsb + 0.008*antonio + 0.008*gb'),
 (7,
  '0.012*azeredo + 0.012*silveira + 0.009*brasilia + 0.009*rio + 0.009*andrade'),
 (8,
  '0.008*grandes + 0.006*brasil + 0.006*bogotá + 0.006*dar + 0.006*parabens'),
 (9, '0.020*lt + 0.017*silveira + 0.014*rio + 0.014*rjo + 0.011*hl'),
 (10,
  '0.020*silveira + 0.018*azeredo + 0.011*antonio + 0.008*brasilia + 0.008*telegrama'),
 (11,
  '0.015*silveira + 0.009*azeredo + 0.009*telegrama + 0.009*df + 0.007*ministro'),
 (12,
  '0.

#### Visualizando os tópicos com o PyLDAvis (45 tópicos)

In [64]:
# https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
6      6.928146        1       1  0.121056 -0.119064
9      6.676816        1       2  0.198664  0.083172
31     4.504779        1       3  0.136061  0.020478
13     4.320999        1       4  0.102188  0.090799
17     3.897992        1       5  0.122940  0.121695
5      3.887790        1       6  0.068778  0.011374
43     3.519939        1       7  0.059636 -0.094146
23     3.441000        1       8  0.102668  0.018246
15     3.359416        1       9  0.093078  0.090578
19     3.269499        1      10  0.085188  0.060690
14     3.251806        1      11  0.060926 -0.128931
33     3.060671        1      12  0.044017 -0.077358
10     2.775136        1      13  0.059037 -0.061350
35     2.753424        1      14  0.055933 -0.046253
44     2.728508        1      15  0.038252 -0.035535
8      2.602026        1      16 -0.078542  0.023023
25     2.572088        1      17  0.008826 -0.057526
24     2.355097        1      18 -0.052490  0.030659
36     2.242509        1      19  0.011335 -0.076566
27     2.213158        1      20  0.021143  0.058483
11     2.197801        1      21  0.021732 -0.063353
30     2.144930        1      22 -0.025617 -0.082464
34     2.078099        1      23 -0.018367 -0.005986
37     1.948439        1      24  0.037105 -0.005215
22     1.860221        1      25 -0.036903 -0.055230
40     1.814436        1      26 -0.050010  0.000270
18     1.808242        1      27 -0.012782  0.034788
20     1.654416        1      28 -0.083705  0.034415
29     1.382019        1      29 -0.079495  0.023290
16     1.382010        1      30 -0.045976  0.004154
7      1.373714        1      31 -0.026374 -0.042166
1      1.300017        1      32 -0.016886 -0.040225
4      1.043983        1      33 -0.039814  0.029035
39     1.014270        1      34 -0.023233  0.064051
0      0.969355        1      35 -0.046034  0.056684
26     0.899191        1      36 -0.065522  0.036094
42     0.826823        1      37 -0.039405 -0.001371
21     0.826817        1      38 -0.088502  0.019505
41     0.804350        1      39 -0.092358  0.017112
3      0.669595        1      40 -0.068363  0.005130
12     0.482138        1      41 -0.088600  0.005279
28     0.362348        1      42 -0.087447  0.017158
38     0.318246        1      43 -0.104280  0.006883
32     0.280014        1      44 -0.071158  0.016405
2      0.197728        1      45 -0.106696  0.013290, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
3448  Default   44.000000                lt   44.000000  30.0000  30.0000
390   Default   38.000000               rjo   38.000000  29.0000  29.0000
4233  Default   33.000000            241gtx   33.000000  28.0000  28.0000
7286  Default   51.000000                gb   51.000000  27.0000  27.0000
2663  Default   90.000000               rio   90.000000  26.0000  26.0000
1700  Default   83.000000           antonio   83.000000  25.0000  25.0000
3652  Default   52.000000             souto   52.000000  24.0000  24.0000
4998  Default   18.000000             paulo   18.000000  23.0000  23.0000
7102  Default   75.000000        embaixador   75.000000  22.0000  22.0000
5319  Default   47.000000             amigo   47.000000  21.0000  21.0000
5474  Default  108.000000           azeredo  108.000000  20.0000  20.0000
1030  Default   50.000000               bsb   50.000000  19.0000  19.0000
1952  Default   55.000000            vieira   55.000000  18.0000  18.0000
3644  Default   24.000000            senhor   24.000000  17.0000  17.0000
5872  Default   28.000000          ministro   28.000000  16.0000  16.0000
6989  Default   29.000000          brasilia   29.000000  15.0000  15.0000
6524  Default   27.000000                hl   27.000000  14.0000  14.0000
3481  Default   20.000000               rua   20.000000  13.

In [65]:
data_ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.save_html(data_ldavis, os.path.join(outputs,'pyldavis_output_45topics.html'))